# 猫狗大战 - 迁移学习的一些尝试

## 1. 模型的训练方式

- 深度学习模型可以划分为 **训练** 和 **预测** 两个阶段。


- **训练** 又分为两种策略。一种是白手起家从头搭建模型进行训练，一种是通过预训练模型进行训练。


- **预测** 相对简单，直接用已经训练好的模型对数据集进行预测即可。（此时采用上面两种策略训练得到的模型都被视为“已训练模型”）

$$
\textbf{Application}
\begin{cases}
    \textbf{Training}
    \begin{cases}
        \color{Blue}{\textbf{Training From Scratch}} \\[2ex]
        \color{Green}{\textbf{Using Pre-trained Model}}
        \begin{cases}
            ^{\#} \textrm{1. Transfer Learning} \\[2ex]
            ^{\#} \textrm{2. Extract Feature Vector (Bottle Neck)} \\[2ex]
            ^{\#} \textrm{3. Fine-tune}
        \end{cases}
    \end{cases}
    \\[3ex]
    \textbf{Inference}: \textrm{Using Pre-trained Model}
\end{cases}
$$

## 2. 迁移学习的三种训练方式


$^{\#} \textbf{1. Transfer Learning}$：冻结预训练模型的全部卷积层，只训练自己定制的全连接层。


$^{\#} \textbf{2. Extract Feature Vector}$：先计算出预训练模型的卷积层对所有训练和测试数据的特征向量，然后抛开预训练模型，只训练自己定制的全连接层。


$^{\#} \textbf{3. Fine-tune}$：冻结预训练模型的部分卷积层（通常是靠近输入的大部分卷积层），训练剩下的卷积层（通常是靠近输出的小部分卷积层）和全连接层。

> “迁移学习” 这个名词既可以广义的指代以上全部三种方式，也可以狭义的指具体第一种方式。

## 3. 三种训练方式的对比

- 第一种和第二种训练得到的模型本质上并没有什么区别，但是第二种的计算复杂度要远远好于第一种。


- 第三种是对前两种方法的补充，以进一步提升模型性能。要注意的是，这种方法**并不一定能对模型有所提升**。


- 本质上来讲，这三种迁移学习的方式都是为了让预训练模型能够胜任新数据集的识别工作，能够让预训练模型原本的特征提取能力得到充分的释放和利用。但是，在此基础上如果想让模型能够达到更低的Loss，那么光靠迁移学习是不够的，靠的更多的还是**模型的结构以及新数据集的数量**。

------

# 开始实验

## 0. 数据集预处理

In [1]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [5]:
height = 299
labels = np.array([0] * 12500 + [1] * 12500)
train = np.zeros((25000, height, height, 3), dtype=np.uint8)
test = np.zeros((12500, height, height, 3), dtype=np.uint8)

for i in tqdm(range(12500)):
    img = cv2.imread('./train/cat/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i] = img[:, :, ::-1]
    
for i in tqdm(range(12500)):
    img = cv2.imread('./train/dog/%s.jpg' % str(i))
    img = cv2.resize(img, (height, height))
    train[i + 12500] = img[:, :, ::-1]

for i in tqdm(range(12500)):
    img = cv2.imread('./test/%s.jpg' % str(i + 1))
    img = cv2.resize(img, (height, height))
    test[i] = img[:, :, ::-1]
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

100%|█████████████████████████████████████████████████████| 12500/12500 [00:26<00:00, 465.45it/s]


Training Data Size = 6.24 GB
Testing Data Size = 3.12 GB


## 1. 实验一：冻结全部卷积层 + 训练自己定制的全连接层

#### 要点

- **使用InceptionV3预训练模型**：该模型当初训练时的输入图像尺寸是299x299，且图像通道顺序为RGB，模型总参数为**两千多万个**。
- **预处理**：按照预训练模型原本的预处理方式对数据进行预处理。
- **基模型**：导入预训练模型，注意只导入卷积层部分，并锁定全部卷积层参数。
- **定制模型**：卷积层之后先接全局平均池化（GAP），再接Dropout，再接分类器，根据分类任务选择输出个数。模型可训练参数只有**两千个**。
- **优化器**：采用较小学习率的SGD。
- **数据准备**：将训练集划分为训练集和验证集。
- **定义回调函数以方便训练**：自动在每代结束保存模型，以val_loss作为监控指标进行早停，训练历史数据同步更新到Tensorboard供可视化。
- **需要用非常小的batch_size进行训练**：这样可以让模型更快更好的收敛。
- 可以看到，虽然卷积层全都已经锁定，但是由于样本依然需要从模型的输入一直计算到输出，因此训练还是比较耗时的，训练五代需要十几分钟。

In [4]:
# Preprocess: Standardization
x = Input(shape=(height, height, 3))
x = Lambda(inception_v3.preprocess_input)(x)

# Base Model: Freeze all conv layers
base_model = InceptionV3(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
for layer in base_model.layers:
    layer.trainable = False

# Customized Classifier
y = Dropout(0.2)(base_model.output)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)

# Full Model: Pre-train Conv + Customized Classifier
model = Model(inputs=base_model.input, outputs=y, name='Transfer_Learning')
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model))

Trainable: 2049, Non-Trainable: 21802784


In [6]:
X_train, X_val, y_train, y_val = train_test_split(train, labels, shuffle=True, test_size=0.2, random_state=42)

In [7]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('transfer_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model.fit(x=X_train, y=y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 162s - loss: 0.1837 - acc: 0.9415 - val_loss: 0.0875 - val_acc: 0.9786
Epoch 2/5
20000/20000 [==============================] - 160s - loss: 0.1214 - acc: 0.9564 - val_loss: 0.0666 - val_acc: 0.9832
Epoch 3/5
20000/20000 [==============================] - 160s - loss: 0.1088 - acc: 0.9599 - val_loss: 0.0617 - val_acc: 0.9828
Epoch 4/5
20000/20000 [==============================] - 159s - loss: 0.1083 - acc: 0.9595 - val_loss: 0.0684 - val_acc: 0.9760
Epoch 5/5
20000/20000 [==============================] - 159s - loss: 0.0995 - acc: 0.9646 - val_loss: 0.0548 - val_acc: 0.9842


## 2. 实验二：导出特征向量，再单独训练分类器

#### 要点

- **预处理**：导出训练集和测试集的特征向量之前，一定要记得按照预训练模型的要求进行预处理，否则导出的特征将不是模型的最佳表现。
- **基模型**：基模型同样由InceptionV3的卷积层部分以及全局平均池化（GAP）构成。
- **导出即预测**：所谓导出，其实就是让基模型直接对训练集和测试集进行预测，只不过预测出的不是类别，而是特征向量（特征图的浓缩版本）。
- **导出需要一定时间**：由于导出需要对数据集的所有图片进行预测，因此通常需要一两分钟才能完成。好的是，一旦完成，就可以完全和CNN拜拜了。
- **新模型的输入是特征向量**：新模型的输入不再是训练集的图像本身，而是经过预训练模型“消化”后的图像特征向量，该向量的第一个维度对应于每一个图像样本，其长度为样本的个数，第二个维度是基模型最后一层每个卷积核输出特征图的平均值，对于InceptionV3来说，第二个维度的长度是2048。
- **划分训练集和验证集**：为了训练时了解模型收敛情况，同样对特征向量划分训练集和验证集。
- **定制新模型**：由于已经导出特征向量，因此接下来只需训练一个输入特征长度为2048的全连接网络即可。
- 同样采用回调函数以及很小的batch_size（16）进行训练。

In [8]:
# Preprocess: Standardization
x = Input(shape=(height, height, 3))
x = Lambda(inception_v3.preprocess_input)(x)

# Base Model: Extract feature vector of both train & test dataset
base_model = InceptionV3(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
train_gap = base_model.predict(train, batch_size=128)
test_gap = base_model.predict(test, batch_size=128)

X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(train_gap, labels, shuffle=True, test_size=0.2, random_state=42)

In [9]:
# Input Shape: (Batch Size, Feature Vector length)
x = Input(shape=(X_train_gap.shape[1],))
y = Dropout(0.2)(x)
y = Dense(1, activation='sigmoid', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 2049, Non-Trainable: 0


In [10]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogVSCat-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train_gap, y=y_train_gap, batch_size=16, epochs=5, validation_data=(X_val_gap, y_val_gap), callbacks=[es, mc, tb])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 3s - loss: 0.0807 - acc: 0.9815 - val_loss: 0.0316 - val_acc: 0.9914
Epoch 2/5
20000/20000 [==============================] - 3s - loss: 0.0264 - acc: 0.9927 - val_loss: 0.0256 - val_acc: 0.9920
Epoch 3/5
20000/20000 [==============================] - 3s - loss: 0.0230 - acc: 0.9929 - val_loss: 0.0238 - val_acc: 0.9924
Epoch 4/5
20000/20000 [==============================] - 3s - loss: 0.0204 - acc: 0.9938 - val_loss: 0.0230 - val_acc: 0.9924
Epoch 5/5
20000/20000 [==============================] - 3s - loss: 0.0197 - acc: 0.9942 - val_loss: 0.0228 - val_acc: 0.9924
